In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import argparse

In [2]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
print("x shape")
print(f"train: {x_train.shape}, val: {x_val.shape}, test: {x_test.shape}")
print("y shape")
print(f"train: {y_train.shape}, val: {y_val.shape}, test: {y_test.shape}")

x shape
train: (40000, 32, 32, 3), val: (10000, 32, 32, 3), test: (10000, 32, 32, 3)
y shape
train: (40000, 1), val: (10000, 1), test: (10000, 1)


In [4]:
class MLP(tf.keras.Model):
    def __init__(self, out_dim, hid_dim, n_layer, act):
        super(MLP, self).__init__()
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.act = act

        self.flatten = tf.keras.layers.Flatten(input_shape=(32, 32))
        self.fc = tf.keras.layers.Dense(self.out_dim)
        self.linears = [
            tf.keras.layers.Dense(self.hid_dim) for i in range(self.n_layer - 1)]

        self.fc2 = tf.keras.layers.Dense(self.out_dim)

        if self.act == 'relu':
            self.act = tf.keras.layers.Activation('relu')

    def call(self, x):
        x = self.flatten(x)
        x = self.act(self.fc(x))
        for fc in self.linears:
            x = self.act(fc(x))
        x = self.fc2(x)
        return x

net = MLP(10, 100, 4, 'relu')

In [5]:
def experiment(args):
    net = MLP(args.out_dim, args.hid_dim, args.n_layer, args.act)

    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(learning_rate=args.lr, momentum=args.mm)
    metrics = tf.keras.metrics.SparseCategoricalAccuracy()

    net.compile(optimizer, loss, metrics)

    result = net.fit(
        x_train, y_train, 
        epochs=args.epochs, 
        validation_data=(x_val, y_val), 
        batch_size=args.batch_size)
    
    test_result = net.evaluate(x_test, y_test)
    
    train_loss = result.history['loss']
    val_acc = result.history['val_sparse_categorical_accuracy']
    val_loss = result.history['val_loss']
    test_acc = test_result[1]

    return train_loss, val_loss, val_acc, test_acc

In [7]:
parser = argparse.ArgumentParser()
args = parser.parse_args('')

args.n_layer = 5
args.out_dim = 10
args.hid_dim = 100
args.act = 'relu'

args.lr = 0.001
args.mm = 0.9
args.epochs = 2
args.batch_size = 128

list_var1 = [4 ,5, 6]
list_var2 = [50, 100, 150]

for var1 in list_var1:
    for var2 in list_var2:
        args.n_layer = var1
        args.hid_dim = var2
        result = experiment(args)
        print(result)

Epoch 1/2
313/313 [==============================] - 2s 5ms/step - loss: 2.3621 - sparse_categorical_accuracy: 0.1013 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.0970
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.1000
([2.3620524406433105, 2.30259108543396], [2.3025832176208496, 2.3025832176208496], [0.09700000286102295, 0.09700000286102295], 0.10000000149011612)
Epoch 1/2
313/313 [==============================] - 2s 5ms/step - loss: 5.2256 - sparse_categorical_accuracy: 0.1011 - val_loss: 5.2046 - val_sparse_categorical_accuracy: 0.1013
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 3.3983 - sparse_categorical_accuracy: 0.1810
([5.225581169128418, 4.7853684425354], [5.204588890075684, 3.3751020431518555], [0.10130000114440918, 0.1850000023841858], 0.1809999942779541)
Epoch 1/2
313/313 [==============================] - 2s 5ms/step - loss: 2.5046 - sparse_categorical_accuracy: 0.09